# Manual Tutorial

This tutorial shows how to setup an optimization problem and solve it in an iterative fashion using everest's ask and tell interface. For this purpose the Himmelblau benchmark function is used. For the automatic optimization have a look at `../benchmarks/001-Himmelblau.ipynb`. Enjoy the ride!

## Imports

In [3]:
import os
import pathlib
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


#pypath = os.path.join(str(pathlib.Path.cwd().parent.parent),"app","lib")

#sys.path.append(pypath)

from everest.domain import Domain
from everest.domain.features import ContinuousInputFeature, ContinuousOutputFeature
from everest.domain.desirability_functions import MinIdentityDesirabilityFunction
from everest.strategies.strategy import RandomStrategy
from everest.strategies.botorch.sobo import BoTorchSoboStrategy



## Setup the optimization problem

First the optimization problem ie. its domain has to be setup.

In [24]:
domain = Domain()

domain.add_feature(ContinuousInputFeature(key="x_1",lower_bound=-4., upper_bound=4.))
domain.add_feature(ContinuousInputFeature(key="x_2",lower_bound=-4., upper_bound=4.))

# as we want to minimize the Himmelblau function we have to use the MinIdentityDesirabilityFunction
desirability_function = MinIdentityDesirabilityFunction(w=1.)
domain.add_feature(ContinuousOutputFeature(key="y",desirability_function=desirability_function))

# show the domain
domain


Domain(input_features=[ContinuousInputFeature(key='x_1', lower_bound=-4.0, upper_bound=4.0), ContinuousInputFeature(key='x_2', lower_bound=-4.0, upper_bound=4.0)], output_features=[ContinuousOutputFeature(key='y', desirability_function=MinIdentityDesirabilityFunction(w=1.0))], constraints=[])

## Generate Training data

We will use Everest's random strategy to sample uniform from the defined domain. In addition we implement the Himmelblau function.

In [25]:
def run_candidate_experiments(candidates:pd.DataFrame, **kwargs):
    candidates.eval("y=((x_1**2 + x_2 - 11)**2+(x_1 + x_2**2 -7)**2)",inplace=True)
    candidates["valid_y"] =1
    return candidates

In [40]:
rs = RandomStrategy.from_domain(domain,use_sobol=False)
candidates, _ = rs.ask(candidate_count = 10)
experiments = run_candidate_experiments(candidates)[domain.experiment_column_names].copy()
experiments

,x_1,x_2,y,valid_y
0,2.891108,2.000749,0.421768,1
1,3.181406,1.999215,1.287439,1
2,2.388869,-2.631800,68.167638,1
3,-0.659233,2.382550,70.890295,1
4,1.490468,-0.066461,108.539717,1
5,3.033699,-3.464592,92.275798,1
6,-0.092261,-1.854830,178.364015,1
7,-3.335610,-0.621683,99.230407,1
8,-1.957456,-3.157584,107.651149,1
9,-2.654095,-3.281897,53.631110,1


### Setup Single Objectiver BO and generate candidates
Tell the experiments the optimizer and ask for candidates.

In [41]:
sobo = BoTorchSoboStrategy.from_domain(domain, acquisition_function = "QNEI")
sobo.tell(experiments)
candidates, _ = sobo.ask(candidate_count=2)
candidates

,x_1,x_2,y_pred,y_sd,y_des
0,2.907812,3.322958,15.404523,41.070571,-15.404523
1,-4.000000,-4.000000,32.597511,44.268601,-32.597511


In the next step you can evaluate the candidates and tell them the optimizer again and ask for the next set of candidates.

In [42]:
experiments = run_candidate_experiments(candidates)[domain.experiment_column_names].copy()
sobo.tell(experiments, replace=False)
candidates, _ = sobo.ask(candidate_count=2)
candidates

,x_1,x_2,y_pred,y_sd,y_des
0,3.802180,0.942373,20.972668,34.356627,-20.972668
1,0.600571,4.000000,68.400896,46.630056,-68.400896


Of course you can do this also in a loop. For this purpose, you the `Study` class.